In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'  # 解决matplotlib报错的问题

In [2]:
import torch
from d2l import torch as d2l

In [ ]:
# 6.4.1 多输入通道
def corr2d_multi_in(X, K):
    # 先遍历X和K的第0维（通道维），然后再把结果加起来
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

In [4]:
X = torch.tensor([[[0,1,2],[3,4,5],[6,7,8]],[[1,2,3],[4,5,6],[7,8,9]]], dtype=torch.float32)
K = torch.tensor([[[0,1],[2,3]],[[1,2],[3,4]]], dtype=torch.float32)
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [5]:
# 6.4.2 多输出通道
def corr2d_multi_in_out(X, K):
    # 迭代第0个维度，每次都对X 执行相关运算
    # 最后所有结果都加到一起
    return torch.stack([corr2d_multi_in(X,k) for k in K])

In [6]:
K = torch.stack((K,K+1,K+2),0)
K.shape

torch.Size([3, 2, 2, 2])

In [7]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [8]:
# 6.4.3 1x1卷积层
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [9]:
X = torch.normal(0, 1, (3, 3, 3),dtype=torch.float32)
K = torch.normal(0, 1, (2, 3, 1, 1),dtype=torch.float32)

Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
print(Y1.shape,Y2.shape)
print(Y1)
print(Y2)
assert(float(torch.abs(Y1-Y2).sum())<1e-6)

torch.Size([2, 3, 3]) torch.Size([2, 3, 3])
tensor([[[-2.2720e-01, -1.3081e+00,  9.7149e-01],
         [-4.3355e-01, -7.6837e-01,  2.9656e-01],
         [ 2.2298e-01,  7.6581e-01, -6.0509e-01]],

        [[-5.3562e+00, -2.1743e+00, -2.0443e+00],
         [ 2.0288e-03, -5.4697e-01, -2.5934e+00],
         [ 1.9186e+00,  5.0683e+00, -3.1617e+00]]])
tensor([[[-2.2720e-01, -1.3081e+00,  9.7149e-01],
         [-4.3355e-01, -7.6837e-01,  2.9656e-01],
         [ 2.2298e-01,  7.6581e-01, -6.0509e-01]],

        [[-5.3562e+00, -2.1743e+00, -2.0443e+00],
         [ 2.0288e-03, -5.4697e-01, -2.5934e+00],
         [ 1.9186e+00,  5.0683e+00, -3.1617e+00]]])
